# Training environment information

In [1]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        1.0G        3.6G        1.2M        8.1G         11G
Swap:            0B          0B          0B


In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-c0914989-ecd2-f474-a28d-11da37782150)


In [3]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz


In [4]:
!lscpu | grep 'Core(s) per socket:'

Core(s) per socket:  1


In [5]:
!nvidia-smi

Tue May 24 20:53:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


Preventing disconnect from Google Colab notebook:
Right-click on the connect button and paste the code to the console of the UI:


```
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click() 
 }
 setInterval(ClickConnect,60000)
```

# Setup

In [7]:
!pip install fastai==2.5.3 -q
!pip freeze

absl-py==1.0.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.4
Babel==2.10.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.4
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2022.5.18.1
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.22.4
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda111==9.4.0
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.30
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
di

In [8]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fastai.vision.all import *
import sys

In [9]:
# Load EB_ML python libraries
# The following libraries are used in this notebook and should be installed in your local machine before running this notebook.
# eb_colab_utils.py
# eb_ml_battery_lib.py
# eb_ml_utils.py

# path to load external *.py files used in this notebook
# Note: in Google Colab virtual machine you shoud copy the files in "/content" folder after BEFORE running this notebook's cell
external_python_file_path="'/.'"
sys.path.append(external_python_file_path)


from eb_ml_colab_utils import get_root_path,copy_model_to_google_drive
from eb_ml_battery_lib import load_soc_dataset,generate_image_files_from_eis
from eb_ml_utils import save_model_weights,build_data_loader,build_and_train_learner,score_model

# Training Configuration

In [10]:
#configuration dictionary
config ={}

# Root working folder (local or Google Drive)
# config['ROOT_DIR'] = get_root_path("batterie")
config['ROOT_DIR'] = get_root_path("batterie")  

# Folder with dataset in CSV format
#config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets"
config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets/EIS-vs-SOC-May2022"

# List of SoC level into dataset
#config['soc_list']=['100','090','080','070','060','050','040','030','020','010']
config['soc_list']=['100','090','080','070','060','050','040','030','020','010']


# Folder to store trained model
#config['MODELS_DIR'] = config['ROOT_DIR']+"/models"
config['MODELS_DIR'] = config['ROOT_DIR']+"/models"

Running on COLAB
Mounted at /gdrive


Source data: battery EIS performed for SoC 100,90,80,70,60,50,40,30,20,10. Same frequnecies for each SoC

Data format:  Re{Zbat}-Im{Zbat}j

```Matlab
prefix = {...
    './Batt1/Meas1/'; --> 1
    './Batt1/Meas2/'; --> 2*
	  './Batt2/Meas1/'; --> 3  
    './Batt2/Meas2/'; --> 4*
    './Batt3/Meas1/'; --> 5
    './Batt3/Meas2/'; --> 6* ...
    './Batt4/Meas1/';
    './Batt5/Meas1/';
    './Batt6/Meas1/';
    './Batt7/Meas1/';
    './Batt8/Meas1/';
    './Batt9/Meas1/';
    './Batt10/Meas1/'; -->13
    };
```

# Data Selection

In [11]:

# Data acquition file to load from dateset folder
battery_list=["02_1","02_2","02_3","02_4","02_5","02_6","02_7","02_8","02_9"]

# Model Training

## Single model Train/Validation split

### Single Model - NO data Augmentation

In [12]:
# Enable/disable image dataset generation
# Image genaration is time and resource comsuming task. 
# You need to generate image just once for every cross validation experiment
# generate_images = False
generate_images = True

# The number of epochs is a hyperparameter that defines the number times that the 
# learning algorithm will work through the entire training dataset
# n_epochs=30
n_epochs=50

# Experiment name prefix
# This prefix will be used in name of files and foldersto store the model and the results of the experiment
# Note1: if you want to run multiple experiments you need to change the name prefix
# Note2: if you want to run in a Colab virtual machine generated images and model will be stored in your Google Drive
experiment_name_prefix="Paper_EIS_single_model_NO_AUGMENTATION"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'


print("battery data acquisitions for training and validation: ")
print(battery_list)
model_name=experiment_name_prefix
print("model name: "+model_name)

config['ExperimentName'] = model_name
config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']
config['TEST_IMAGES_PATH'] = config['ROOT_DIR']+"/test_"+config['ExperimentName']

#GENERATE IMAGE
if(generate_images):
    dataset,feature_col_names=load_soc_dataset(battery_list,config["soc_list"],config['DATASETS_DIR'])
    generate_image_files_from_eis(dataset,feature_col_names,config['IMAGES_PATH'],config['ExperimentName'],rescale=False,DATA_AUGMENTATION_FACTOR=1)

# TRAINING
learn= build_and_train_learner(config,n_epochs=n_epochs)
#SAVE
weights_filename=save_model_weights(learn,config["MODELS_DIR"],model_name)
filename_pth= weights_filename+".pth"
copy_model_to_google_drive(filename_pth,learn.model_dir,config["MODELS_DIR"])
# SCORE MODEL
dl=build_data_loader(config)

battery data acquisitions for training and validation: 
['02_1', '02_2', '02_3', '02_4', '02_5', '02_6', '02_7', '02_8', '02_9']
model name: Paper_EIS_single_model_NO_AUGMENTATION
dataset row number: 90
start image file generation. IMAGE_PATH: /gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION
df_real shape: (90, 14)
df_img shape: (90, 14)
soc: 100
battery: 02_1
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/EIS_BATT02_1_100.png
soc: 090
battery: 02_1
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/EIS_BATT02_1_090.png
soc: 080
battery: 02_1
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/EIS_BATT02_1_080.png
soc: 070
battery: 02_1
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUG

Valley: 8.32e-04


epoch,train_loss,valid_loss,accuracy,time
0,3.873679,3.441175,0.074074,00:01


epoch,train_loss,valid_loss,accuracy,time
0,4.049129,2.999057,0.074074,00:01
1,4.140858,2.768130,0.074074,00:01
2,4.065722,2.659856,0.074074,00:01
3,3.986834,2.557322,0.074074,00:01
4,3.996785,2.546986,0.111111,00:01
5,3.909436,2.571549,0.222222,00:01
6,3.758537,2.603920,0.222222,00:01
7,3.711617,2.713374,0.222222,00:01
8,3.682416,2.793834,0.222222,00:01
9,3.617260,2.884445,0.259259,00:01


saved filename: Paper_EIS_single_model_NO_AUGMENTATION_1653425758.821224_SAVED.pth
Setting-up type transforms pipelines
Found 90 items
2 datasets of sizes 63,27
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/EIS_BATT02_7_070.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/EIS_BATT02_7_070.png
    applying RegexLabeller gives
      070
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(6)

Final sample: (PILImage mode=RGB size=432x288, Tens

In [13]:
score_model(weights_filename,dl)

[[2 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1]
 [0 0 1 1 0 0 0 0 0 1]
 [0 0 1 1 0 0 0 0 0 2]
 [0 1 1 1 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 3 1 1]
 [0 0 0 0 0 0 1 1 0 0]
 [1 0 0 0 0 0 0 0 1 1]
 [1 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.5        0.         0.         0.         0.
  0.         0.         0.         0.5       ]
 [0.         0.         0.33333333 0.33333333 0.         0.
  0.         0.         0.         0.33333333]
 [0.         0.         0.25       0.25       0.         0.
  0.         0.         0.         0.5       ]
 [0.         0.25       0.25       0.25       0.         0.
  0.         0.         0.         0.25      ]
 [0.         0.         0.         0.         0.         0.
  0.         0.6        0.2        0.2       ]
 [0.         0.         0.         0.         0.         0.
  0.5        0.5        0.         0.        ]
 [0.33333333 0.     

/content/eb_ml_utils.py:128: RuntimeWarning: invalid value encountered in true_divide
  cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
/usr/local/lib/python3.7/dist-packages/fastai/interpret.py:71: RuntimeWarning: invalid value encountered in true_divide
  if normalize: cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


interpretation.print_classification_report()
              precision    recall  f1-score   support

         010       0.50      1.00      0.67         2
         020       0.50      0.50      0.50         2
         030       0.33      0.33      0.33         3
         040       0.33      0.25      0.29         4
         050       0.00      0.00      0.00         4
         060       0.00      0.00      0.00         5
         070       1.00      0.50      0.67         2
         080       0.00      0.00      0.00         3
         090       0.33      0.50      0.40         2
         100       0.00      0.00      0.00         0

    accuracy                           0.26        27
   macro avg       0.30      0.31      0.29        27
weighted avg       0.26      0.26      0.24        27

learn.show_results()


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

learn.validate()


Model accuracy: 0.26
learn.get_preds()


TensorBase(0.2593)

### Single model - Data Augmentation Very low noise

In [14]:
# Enable/disable image dataset generation
# Image genaration is time and resource comsuming task. 
# You need to generate image just once for every cross validation experiment
# generate_images = False
generate_images = True

# The number of epochs is a hyperparameter that defines the number times that the 
# learning algorithm will work through the entire training dataset
# n_epochs=30
n_epochs=50

# Experiment name prefix
# This prefix will be used in name of files and foldersto store the model and the results of the experiment
# Note1: if you want to run multiple experiments you need to change the name prefix
# Note2: if you want to run in a Colab virtual machine generated images and model will be stored in your Google Drive
experiment_name_prefix="Paper_EIS_single_model_LOW_NOISE"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'


print("battery data acquisitions for training and validation: ")
print(battery_list)
model_name=experiment_name_prefix
print("model name: "+model_name)

config['ExperimentName'] = model_name
config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']
config['TEST_IMAGES_PATH'] = config['ROOT_DIR']+"/test_"+config['ExperimentName']

#GENERATE IMAGE
if(generate_images):
    dataset,feature_col_names=load_soc_dataset(battery_list,config["soc_list"],config['DATASETS_DIR'])
    generate_image_files_from_eis(dataset,feature_col_names,config['IMAGES_PATH'],config['ExperimentName'],rescale=False,NOISE_AMOUNT=1e-12,DATA_AUGMENTATION_FACTOR=10)

# TRAINING
learn= build_and_train_learner(config,n_epochs=n_epochs)
#SAVE
weights_filename=save_model_weights(learn,config["MODELS_DIR"],model_name)
filename_pth= weights_filename+".pth"
copy_model_to_google_drive(filename_pth,learn.model_dir,config["MODELS_DIR"])
# SCORE MODEL
dl=build_data_loader(config)

battery data acquisitions for training and validation: 
['02_1', '02_2', '02_3', '02_4', '02_5', '02_6', '02_7', '02_8', '02_9']
model name: Paper_EIS_single_model_LOW_NOISE
dataset row number: 90
start image file generation. IMAGE_PATH: /gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE
df_real shape: (90, 14)
df_img shape: (90, 14)
soc: 100
battery: 02_1
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/EIS_BATT02_1_100.png
augmentation_index: 1
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/EIS_BATT1100_100.png
augmentation_index: 2
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/EIS_BATT1200_100.png
augmentation_index: 3
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/EIS_BATT1300_100.png
augmentation_index: 4
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_m

Valley: 1.00e-03


epoch,train_loss,valid_loss,accuracy,time
0,3.286809,2.134205,0.251852,00:07


epoch,train_loss,valid_loss,accuracy,time
0,2.342803,1.866050,0.392593,00:07
1,2.203429,1.317686,0.507407,00:07
2,1.916024,1.029059,0.574074,00:07
3,1.711387,0.845464,0.662963,00:07
4,1.546518,0.633036,0.796296,00:07
5,1.383533,0.492660,0.885185,00:07
6,1.217125,0.467601,0.837037,00:07
7,1.069738,0.428524,0.851852,00:07
8,0.927764,0.312731,0.900000,00:07
9,0.812270,0.178311,0.981481,00:07


saved filename: Paper_EIS_single_model_LOW_NOISE_1653426219.972353_SAVED.pth
Setting-up type transforms pipelines
Found 900 items
2 datasets of sizes 630,270
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/EIS_BATT1767_030.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/EIS_BATT1767_030.png
    applying RegexLabeller gives
      030
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(2)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(2))


Found 900 

In [15]:
score_model(weights_filename,dl)

[[20  0  0  0  0  0  0  0  0  0]
 [ 0 28  0  0  0  0  0  0  0  0]
 [ 0  0 29  0  0  0  0  0  0  0]
 [ 0  0  0 22  0  0  0  0  0  0]
 [ 0  0  0  0 25  0  0  0  0  0]
 [ 0  0  0  0  0 32  0  0  0  0]
 [ 0  0  0  0  0  0 29  0  0  0]
 [ 0  0  0  0  0  0  0 29  0  0]
 [ 0  0  0  0  0  0  0  0 28  0]
 [ 0  0  0  0  0  0  0  0  0 28]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
interpretation.most_confused()
interpretation.top_losses()
interpretation.print_classification_report()
              precision    recall  f1-score   support

         010       1.00      1.00      1.00        20
         020       1.00      1.00      1.00        28
         030       1.00      1.00      1.00        29
         040     

learn.validate()


Model accuracy: 1.00
learn.get_preds()


TensorBase(1.)

### Single Model - Data augmentation with realistic AGW noise amount in data acquisition simulation

In [16]:
# Enable/disable image dataset generation
# Image genaration is time and resource comsuming task. 
# You need to generate image just once for every cross validation experiment
# generate_images = False
generate_images = True

# The number of epochs is a hyperparameter that defines the number times that the 
# learning algorithm will work through the entire training dataset
# n_epochs=30
n_epochs=50

# Experiment name prefix
# This prefix will be used in name of files and foldersto store the model and the results of the experiment
# Note1: if you want to run multiple experiments you need to change the name prefix
# Note2: if you want to run in a Colab virtual machine generated images and model will be stored in your Google Drive
experiment_name_prefix="Paper_EIS_single_model"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'


print("battery data acquisitions for training and validation: ")
print(battery_list)
model_name=experiment_name_prefix
print("model name: "+model_name)

config['ExperimentName'] = model_name
config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']
config['TEST_IMAGES_PATH'] = config['ROOT_DIR']+"/test_"+config['ExperimentName']

#GENERATE IMAGE
if(generate_images):
    dataset,feature_col_names=load_soc_dataset(battery_list,config["soc_list"],config['DATASETS_DIR'])
    generate_image_files_from_eis(dataset,feature_col_names,config['IMAGES_PATH'],config['ExperimentName'],rescale=False,DATA_AUGMENTATION_FACTOR=10)

# TRAINING
learn= build_and_train_learner(config,n_epochs=n_epochs)
#SAVE
weights_filename=save_model_weights(learn,config["MODELS_DIR"],model_name)
filename_pth= weights_filename+".pth"
copy_model_to_google_drive(filename_pth,learn.model_dir,config["MODELS_DIR"])
# SCORE MODEL
dl=build_data_loader(config)

battery data acquisitions for training and validation: 
['02_1', '02_2', '02_3', '02_4', '02_5', '02_6', '02_7', '02_8', '02_9']
model name: Paper_EIS_single_model
dataset row number: 90
start image file generation. IMAGE_PATH: /gdrive/MyDrive/batterie/Paper_EIS_single_model
df_real shape: (90, 14)
df_img shape: (90, 14)
soc: 100
battery: 02_1
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT02_1_100.png
augmentation_index: 1
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT1100_100.png
augmentation_index: 2
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT1200_100.png
augmentation_index: 3
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT1300_100.png
augmentation_index: 4
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT1400_100.png
augmentation_index: 5
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_sing

Valley: 1.20e-03


epoch,train_loss,valid_loss,accuracy,time
0,3.246968,2.414026,0.159259,00:07


epoch,train_loss,valid_loss,accuracy,time
0,2.582578,2.087217,0.300000,00:07
1,2.365107,1.683057,0.414815,00:07
2,2.247171,1.425990,0.470370,00:07
3,2.040946,1.303186,0.485185,00:07
4,1.892256,1.178599,0.570370,00:07
5,1.752327,1.121668,0.592593,00:07
6,1.615196,1.096795,0.611111,00:07
7,1.517413,1.057534,0.640741,00:07
8,1.421474,1.048335,0.614815,00:07
9,1.302854,0.938935,0.674074,00:07


saved filename: Paper_EIS_single_model_1653426688.521407_SAVED.pth
Setting-up type transforms pipelines
Found 900 items
2 datasets of sizes 630,270
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT1767_030.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT1767_030.png
    applying RegexLabeller gives
      030
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(2)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(2))


Found 900 items
2 datasets of sizes 630,270
Setting up Pipel

In [17]:
score_model(weights_filename,dl)

[[20  0  0  0  0  0  0  0  0  0]
 [ 0 27  1  0  0  0  0  0  0  0]
 [ 0  1 20  2  0  0  0  0  0  6]
 [ 0  0  1 20  1  0  0  0  0  0]
 [ 0  0  1  1 22  0  0  0  0  1]
 [ 0  0  0  0  0 29  0  0  3  0]
 [ 0  0  0  0  0  0 21  7  1  0]
 [ 0  0  0  0  0  1  3 24  1  0]
 [ 0  0  0  0  0  0  0  1 27  0]
 [ 0  2  3  0  0  0  0  0  0 23]]
[[1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.96428571 0.03571429 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.03448276 0.68965517 0.06896552 0.         0.
  0.         0.         0.         0.20689655]
 [0.         0.         0.04545455 0.90909091 0.04545455 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.04       0.04       0.88       0.
  0.         0.         0.         0.04      ]
 [0.         0.         0.         0.         0.         0.90625
  0.         0.         0.09375    0.        ]
 [0.         0.       

learn.validate()


Model accuracy: 0.86
learn.get_preds()


TensorBase(0.8630)